# Decision Tree 결정트리

1. 분류, 회귀, 다중출력 등 다재다능한 머신러닝 알고리즘
2. Random forest의 기본 구성요소
3. 전처리가 필요하지않다
4. 과정을 설명하기 쉽고 시각화하기 쉽다

In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris=load_iris()
X=iris.data[:,2:]
y=iris.target

tree_clf=DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(max_depth=2)

In [8]:
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

In [3]:
from graphviz import Source
from sklearn.tree import export_graphviz

# export_graphviz(
#     tree_clf,
#     out_file=os.path.join(IMAGES_PATH, "iris_tree.dot"),
#     feature_names=iris.feature_names[2:],
#     class_names=iris.target_names,
#     rounded=True,
#     filled=True
# )
Source.from_file("images/decision_trees/iris_tree.dot")

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

## 예측하기

- 노드 속성 : gini, samples, value, class

- samples 속성이란? 얼마나 많은 train data sample가 들어갔는가? 
노드를 시각화하면 각 조건에 따라 몇 개의 sample이 조건하에 들어갔는지 알 수 있다.

- value ? 각 클래스별 훈련 샘플 갯수

- gini ? 불순도 측정, all sample == 하나의 class 라면 순수(gini=0)이다. 지니 불순도 식을 이용해 불순도 측정 가능.

### 화이트박스 & 블랙박스
화이트박스 :
- 결정 트리 Decision Tree
- 예측과정을 직접 수동으로 따라 해볼 수 있다

블랙박스 :
- Random forest, 신경망
- 예측과정을 설명하기 힘듦

## 결정 트리의 클래스 확률 추정
한 샘플이 특정 클래스에 속한 확률을 추정한다.
1. leaf node를 찾기 위해 트리를 탐색
2. 해당 노드의 클래스 k의 훈련 샘플의 비율 return
3. 가장 높은 확률을 가진 클래스를 예측 클래스로 출력

## CART 훈련 알고리즘
scikit-learn 에서 Decision Tree로 training을 위해 CART 알고리즘을 쓴다. \
탐욕적 알고리즘 greedy algorithm. \
평균제곱오차 MSE를 최소화하도록 분할 (not train dataset 불순도를 최소화하는 방향으로 분할)


1. 가장 순수히 서브셋을 나눌 수 있는 (k, k임계값) 짝을 찾는다.
2. train dataset을 하나의 특성 k의 임곗값으로 두 개의 서브셋으로 나눈다.
(CART 비용함수 참고)
3. 같은 방식으로 서브셋을 또 나누고 ... 반복
4. max_depth 정의된만큼 되면 중지 or 불순도를 줄이는 분할을 찾을 수 없을 때 stop
5. 중지할 수 있는 조건들 (min_samples_split, nim_samples_leaf, min_weight_fraction_leaf, max_leaf_nodes)

## 계산 복잡도
예측 과정 - root node ~ leaf node까지 결정 트리를 탐색한다.

약 O(log_2(m))개 노드를 거친다. \
각 노드에서 모든 샘플, 모든 특성을 탐색시 복잡도 : O(nxmlog_2(m)) 

여기서 n은 feature 갯수, max_features로 지정해 줄수도 있다. \
train datset이 크면 속도가 많이 느려진다.

수천 개 이하의 훈련세트는 사이킷런에서 presort=True로 설정하여 속도를 높일 수있다. 하지만 데이터셋이 크면 데이터를 정렬하는 작업이 추가되기 때문에 속도가 더 느려진다.

## 지니 불순도 OR ENTROPY 엔트로피
criterion="entropy" 매개변수 지정하여 엔트로피 불순도 사용가능 
- 분자의 무질서함을 측정
- 분자가 안정 & 질서 정연 -> entropy 는 0에 가깝다
- 예) 모든 메시지가 동일 => entropy=0
- 엔트로피 식 참조

지니 불순도와 엔트로피
- 큰 차이는 없다
- 지니 불순도가 좀 더 빠름
- 지니 불순도는 가장 빈도 높은 클래스를 한쪽으로 고립시키기도함
- 비교적 엔트로피가 좀 더 균형 잡힌 트리를 만듦

## 성능 높이기
### 방법1. 규제 매개변수
1. Decision Tree는 제약 사항이 거의 없다.
2. 제약이 없으면 과대적합이 잘 된다.
3. Decision Tree는 훈련전 파라미터 수가 결정되지 않는다 - 비파라미터 모델
4. 비파라미터 모델 : 데이터에 모델구조가 맞춰진다. 모델이 고정되지 않고 자유로움.
5. 파라미터 모델 : 자유도가 제한되고 과대적합 위험이 줄어든다.
6. 과대적합 위험을 줄이기 위해 Decision Tree에 자유도를 제한하는 규제를 적용한다.
7. Decision Tree의 최대 깊이 max depth 제어부터 가능하다.
8. 기타 매개변수들) min_samples_split, min_samples_leaf, mi_weight_fraction_leaf ... 

### 방법2. 가지치기
1. 불필요한 노드를 제거하는 방법
2. 어떤 노드가 불필요한지 판단하는 방법 : chi-squared-test(p-value)를 통해 바로 위의 노드가 의미있는지 확인한다 -> p-value가 임곗값보다 높으면 해당 노드의 자식노드를 삭제한다.  -> 불필요한게 없어질 때 까지 반복

## 회귀 

In [7]:
from sklearn.tree import DecisionTreeClassifier

tree_reg=DecisionTreeClassifier(max_depth=2)
tree_reg(X,y)

TypeError: 'DecisionTreeClassifier' object is not callable

## 불안정성
1. DecisionTree 는 계단식 결정 경계Decision boundary를 만든다.
2. So, train datset 회전, 작은 변화에 아주 민감하다.
3. 회전된 데이터셋을 사용해 훈련하면 일반화되기 쉽지 않다.
4. 이 해결책은 훈련 데이터셋을 질 좋게 회전시키는 PCA 기법을 사용하는 것이다.
5. Decision Tree의 이러한 문제를 Random Forest에서 예측 평균치로 극복가능하다고 한다.



> 연습문제1. 백만 개의 샘플을 가진 훈련 세트에서 규제없이 훈련시킨 결정 트리의 깊이는 대략 얼마일까요? 

트리의 깊이 : log_2(m) 이므로, log_2(백만개)

> 연습문제2. 한 노드의 지니 불순도가 보통 그 부모 노드보다 작을까요, 아니면 클까요? 일반적으로 작거나 클까요, 아니면 항상 작거나 클까요?

한 노드의 지니 불순도는 부모 불순도보다 늘 낮다. 지니 불순도의 가중치 합이 최소화되는 CART 훈련 알고리즘 비용 함수를 사용하기 때문에, 불순도를 계속 낮추는 방향으로 작용한다. 

> 연습문제3. 결정 트리가 훈련 세트에 과대적합되었다면 max_depth를 줄이는 것이 좋을까?

max_depth를 적절히 줄여 규제를 추가하는 것이 좋다. 

> 연습문제4. 결정 트리가 훈련 세트에 과소적합되었다면 입력 특성의 스케일을 조정하는 것이 좋을까? 

결정 트리는 전처리가 영향을 크게 주지 않는다. 과소적합되었다면 데이터셋을 더 추가하는것이 좋다. max_depth에 따라 몇 개의 클래스로 분류하는지 결정되는데, 너무 적게하면 과소적합, 너무 많게하면 과대적합으로 이어질 수 있다. max_depth, max_leaf_nodes, max_samples_leaf 매개변수를 조절해볼 수 있겠다.

> 연습문제5. 백만 개의 샘플을 가진 훈련 세트에 결정 트리를 훈련시키는 데 한 시간이 걸렸다면 천만개의 샘플을 가진 훈련 세트에 결정 트리를 훈련시키는 데는 대략 얼마나 걸릴까?

O(n x 백만개log_2(백만개)) = 한 시간
O(n x 천만개log_2(천만개)) = ?

K=(n×10m×log(10m))/(n×m×log(m))=10×log(10m)/log(m) \
 m=106 
 이면, k= 11.7시간 정도이다.

> 연습문제6. 십만 개의 샘플을 가진 훈련 세트가 있다면 presort=True로 지정하는 것이 훈련 속도를 높일까요?

아니요, 수천 개 이하의 샘플에만 적용이 됩니다. 정렬하는 과정이 추가되기 때문에 데이터셋이 많다면 속도가 더 느려집니다. 

> 연습문제7. 다음 단계를 따라 moons 데이터셋에 결정 트리를 훈련시키고 세밀하게 튜닝해보세요.

> a. make_moons(n_samples=1000, noise=0.4)를 사용해 데이터셋을 생성합니다.

In [9]:
from sklearn.datasets import make_moons

X,y = make_moons(n_samples=1000, noise=0.4, random_state=42)

> b. 이름 train_test_split()을 사용해 훈련, 테스트 세트로 나눕니다.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

> c. DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 탐색을 수행합니다(GridSearchCV쓰면됨) 여러가지 max_leaf_nodes 값 시도해볼 것

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# model 선언
tree_clf=DecisionTreeClassifier(random_state=42)

# 파라미터값 선언
params={'max_leaf_nodes':list(range(2,50)), 'min_samples_leaf':[2,3,4]}

# Gridsearch
grid_search = GridSearchCV(estimator=tree_clf, param_grid=params, cv=3, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 144 candidates, totalling 432 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_leaf': [2, 3, 4]},
             verbose=1)

In [24]:
grid_search.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=4, min_samples_leaf=2, random_state=42)

In [25]:
from sklearn.metrics import accuracy_score

y_hat=grid_search.best_estimator_.predict(X_test)
accuracy_score(y_test, y_hat)

0.855

> 연습문제8. 다음 단계를 따라 랜덤 포레스트를 만들어보세요.

a. 훈련 세트의 서브셋 1000개 생성. 무작위로 선택된 100개의 샘플. 

In [44]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

X_train 중 임의의 100개의 인덱스값만 뽑아 mini_sets 데이터셋을 만든다.

In [45]:
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train=X_train[mini_train_index]
    y_mini_train=y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b. 이전 연습문제에서 찾은 최적의 매개변수로 각 서브셋에 결정 트리를 훈련시킨다. 테스트 세트로 이 1000개의 결정 트리를 평가한다. (더 작은 데이터셋에서 훈련해서 앞서 만든 결정트리 모델보다 성능이 떨어져 약 80% 정확도가 나올 것)

In [46]:
# mini_tree_clf=DecisionTreeClassifier(random_state=42)
""" model을 새로 만들어줄 필요없이 clone을 이용해서 
앞서 만든 model을 복사해오면 된다. """
from sklearn.base import clone

forest = [clone(grid_search.best_estimator_) for _ in range(n_trees)]

In [52]:
y_pred

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0])

In [67]:
import numpy as np
from sklearn.metrics import accuracy_score

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    
    # print(accuracy_score(y_test, y_pred))
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8160749999999999

c. 각 테스트 세트 샘플에 대해 1000개의 결정 트리 예측을 만들고,
다수로 나온 예측만 추려줍니다. 테스트 세트에 대한 다수결 예측을 만듭니다.

In [68]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)


from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

y_pred_majority_votes : Return an array of the modal (most common) value in the passed array.  \
: What is 'most commmon value? 단순히 가장 많은 값에서 부터 값이 동일 하면 가장 앞에 있는 값을 선택해서 보여준다.

n_votes :  The bin-count for the modal bins is also returned. \
: common value가 몇 번 나왔는지 세어준다.


d. 테스트 세트에서 이 예측을 평가합니다. 랜덤 포레스트 분류기를 훈련을 완성했습니다!

In [71]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.85